In [ ]:
import tkinter as tk
from tkinter import messagebox, font
import pandas as pd
import matplotlib.pyplot as plt

# Main Window Setup
root = tk.Tk()
root.title("BMI Calculator")

# Increase window size
root.geometry("400x300")

# Define larger font
large_font = font.Font(size=14)

# Input Fields
tk.Label(root, text="Name:", font=large_font).grid(row=0, column=0, pady=10, padx=10, sticky='e')
name_entry = tk.Entry(root, font=large_font)
name_entry.grid(row=0, column=1, pady=10, padx=10)

tk.Label(root, text="Weight (kg):", font=large_font).grid(row=1, column=0, pady=10, padx=10, sticky='e')
weight_entry = tk.Entry(root, font=large_font)
weight_entry.grid(row=1, column=1, pady=10, padx=10)

tk.Label(root, text="Height (m):", font=large_font).grid(row=2, column=0, pady=10, padx=10, sticky='e')
height_entry = tk.Entry(root, font=large_font)
height_entry.grid(row=2, column=1, pady=10, padx=10)

# Result Display
bmi_result = tk.StringVar()
result_label = tk.Label(root, textvariable=bmi_result, font=large_font)
result_label.grid(row=4, columnspan=2, pady=10, padx=10)

# Functions
def validate_inputs(weight, height):
    try:
        weight = float(weight)
        height = float(height)
        if weight <= 0 or height <= 0:
            raise ValueError
        return weight, height
    except ValueError:
        messagebox.showerror("Input Error", "Please enter valid, positive numbers for weight and height")
        return None, None

def calculate_bmi(weight, height):
    return weight / (height ** 2)

def categorize_bmi(bmi):
    if bmi < 18.5:
        return "Underweight"
    elif 18.5 <= bmi < 24.9:
        return "Normal weight"
    elif 25 <= bmi < 29.9:
        return "Overweight"
    else:
        return "Obesity"

data_file = "bmi_data.csv"

def save_data(name, weight, height, bmi, category):
    try:
        global data
        new_data = pd.DataFrame([[name, weight, height, bmi, category, pd.Timestamp.now()]], columns=["Name", "Weight", "Height", "BMI", "Category", "Date"])
        data = pd.concat([data, new_data], ignore_index=True)
        data.to_csv(data_file, index=False)
    except Exception as e:
        messagebox.showerror("Data Error", f"Failed to save data: {e}")

def load_data():
    global data
    try:
        data = pd.read_csv(data_file)
    except FileNotFoundError:
        data = pd.DataFrame(columns=["Name", "Weight", "Height", "BMI", "Category", "Date"])

def plot_trend():
    user_data = data[data["Name"] == name_entry.get()]
    if user_data.empty:
        messagebox.showinfo("No Data", "No data available for this user.")
        return
    user_data.plot(x="Date", y="BMI", title="BMI Trend", marker='o')
    plt.show()

def on_calculate():
    weight, height = validate_inputs(weight_entry.get(), height_entry.get())
    if weight and height:
        bmi = calculate_bmi(weight, height)
        category = categorize_bmi(bmi)
        bmi_result.set(f"BMI: {bmi:.2f} - {category}")
        save_data(name_entry.get(), weight, height, bmi, category)

# Buttons
tk.Button(root, text="Calculate BMI", command=on_calculate, font=large_font).grid(row=3, columnspan=2, pady=10, padx=10)
tk.Button(root, text="Show Trend", command=plot_trend, font=large_font).grid(row=5, columnspan=2, pady=10, padx=10)

# Load data at the start
load_data()

# Start the main loop
root.mainloop()
